In [ ]:
import pandas as pd
from pylab import *
from datetime import datetime
import xgboost as xgb

import du fichier train

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

data = pd.read_csv('gdrive/My Drive/train_V2.csv')

-suppression des mauvais données et des colonnes trop corrélés  
-renommage des différents types de matchs pour avoir que les 3 souhaités

In [ ]:
data = data.dropna(axis=0, subset=['winPlacePerc'])
data = data.loc[(data['killPlace'] < 101)]
data = data.drop(["DBNOs","maxPlace","winPoints"], axis = 1)

data.loc[(data['matchType']== 'squad-fpp'),'matchType']='squad'
data.loc[(data['matchType']== 'normal-squad-fpp'),'matchType']='squad'
data.loc[(data['matchType']== 'normal-squad'),'matchType']='squad'
data.loc[(data['matchType']== 'flarefpp'),'matchType']='squad'
data.loc[(data['matchType']== 'flaretpp'),'matchType']='squad'

data.loc[(data['matchType']== 'duo-fpp'),'matchType']='duo'
data.loc[(data['matchType']== 'normal-duo-fpp'),'matchType']='duo'
data.loc[(data['matchType']=='normal-duo'),'matchType']='duo'
data.loc[(data['matchType']=='crashfpp'),'matchType']='duo'
data.loc[(data['matchType']=='crashtpp'),'matchType']='duo'

data.loc[(data['matchType']== 'solo-fpp'),'matchType']='solo'
data.loc[(data['matchType']== 'normal-solo-fpp'),'matchType']='solo'
data.loc[(data['matchType']== 'normal-solo'),'matchType']='solo'

-kmeans et ajout de la classe dans les données

In [ ]:
from sklearn.cluster import KMeans
datak = data.drop(["Id","groupId","matchId","matchType"], axis = 1)
kmeans = KMeans(n_clusters=7, random_state=0,max_iter=10,n_init=1).fit(datak)
data['classe']=kmeans.labels_

-création de variables pouvant être intéressantes

In [ ]:
data['healParDurationM']=data['heals']/data['matchDuration']
data['killParDurationM']=data['kills']/data['matchDuration']
data['killParRideDist']=data['kills'].divide(data['rideDistance'])
data['RideDistParDurationM']=data['rideDistance']/data['matchDuration']
data['BoostParKill']=data['boosts'].divide(data['kills'])
data['KillParSwimDist']=data['kills'].divide(data['swimDistance'])
data['KillParWalkDist']=data['kills'].divide(data['walkDistance'])

-création des moyennes et des maximums par match et par groupe  
-création de 2 variables de comparaison par rapport au match

In [ ]:
ok=data.drop(["matchType","Id","groupId","classe","winPlacePerc"], axis = 1)
mean=ok.groupby('matchId').mean()
maxi=ok.groupby('matchId').max()


mean.columns = [x + "meanMatch" for x in mean.columns]
maxi.columns = [x + "maxMatch" for x in maxi.columns]


meanG=ok.groupby('groupId').mean()
maxiG=ok.groupby('groupId').max()


meanG.columns = [x + "meanGroupe" for x in meanG.columns]
maxiG.columns = [x + "maxGroupe" for x in maxiG.columns]


data=data.merge(meanG,on='groupId').merge(maxiG,on='groupId').merge(mean,on='matchId').merge(maxi,on='matchId')


data = pd.get_dummies(data, columns=['classe'])
data=data.replace([inf], np.nan)
data=data.fillna(0)

data['DiffKillMaxKill']=data['kills']-data['killsmaxMatch']
data['DiffDmgMaxDmg']=data['damageDealt']-data['damageDealtmaxMatch']


solo=data[(data.matchType=='solo')]
duo=data[(data.matchType=='duo')]
squad=data[(data.matchType=='squad')]

-suppression des dataframes inutiles

In [ ]:
import gc
del [[mean,maxi,meanG,maxiG,ok,datak]]
gc.collect()

# Partie modélisation

## Modèle global

In [ ]:
data = data.drop(["Id","groupId","matchId"], axis = 1)
data = pd.get_dummies(data, columns=['matchType'])
from sklearn.model_selection import train_test_split
feat=data.iloc[:,np.r_[0:21, 22:153]]
target=data.iloc[:,[21]]

-on commence par séparer les données d'apprentissage de test selon la répartition souhaitée  
-puis on vient diviser les données d'apprentissage en 10 échantillons

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.datasets import load_boston
from sklearn.metrics import mean_absolute_error as mse

rs = ShuffleSplit(test_size=0.001, n_splits=1, random_state=0)
for train_idx,test_idx in rs.split(feat):  
    pass


train_split = round(len(train_idx) / 10)
train1_idx = train_idx[0:train_split]
train2_idx = train_idx[train_split:(2*train_split)]
train3_idx = train_idx[(2*train_split):(3*train_split)]
train4_idx = train_idx[(3*train_split):(4*train_split)]
train5_idx = train_idx[(4*train_split):(5*train_split)]
train6_idx = train_idx[(5*train_split):(6*train_split)]
train7_idx = train_idx[(6*train_split):(7*train_split)]
train8_idx = train_idx[(7*train_split):(8*train_split)]
train9_idx = train_idx[(8*train_split):(9*train_split)]
train10_idx = train_idx[(9*train_split):(10*train_split)]

X_train_1 = feat.loc[train1_idx]
X_train_2 = feat.loc[train2_idx]
X_train_3 = feat.loc[train3_idx]
X_train_4 = feat.loc[train4_idx]
X_train_5 = feat.loc[train5_idx]
X_train_6 = feat.loc[train6_idx]
X_train_7 = feat.loc[train7_idx]
X_train_8 = feat.loc[train8_idx]
X_train_9 = feat.loc[train9_idx]
X_train_10 = feat.loc[train10_idx]
X_test=feat.loc[test_idx]

y_train_1 = target.loc[train1_idx]
y_train_2 = target.loc[train2_idx]
y_train_3 = target.loc[train3_idx]
y_train_4 = target.loc[train4_idx]
y_train_5 = target.loc[train5_idx]
y_train_6 = target.loc[train6_idx]
y_train_7 = target.loc[train7_idx]
y_train_8 = target.loc[train8_idx]
y_train_9 = target.loc[train9_idx]
y_train_10 = target.loc[train10_idx]
y_test=target.loc[test_idx]

In [ ]:
import gc
del [[data,feat,target]]
gc.collect()

-on place les données d'apprentissage et de test dans des objets DMatrix pour être utilisées dans les modèles

In [ ]:
xg_train_1 = xgb.DMatrix(X_train_1, label=y_train_1)
xg_train_2 = xgb.DMatrix(X_train_2, label=y_train_2)
xg_train_3 = xgb.DMatrix(X_train_3, label=y_train_3)
xg_train_4 = xgb.DMatrix(X_train_4, label=y_train_4)
xg_train_5 = xgb.DMatrix(X_train_5, label=y_train_5)
xg_train_6 = xgb.DMatrix(X_train_6, label=y_train_6)
xg_train_7 = xgb.DMatrix(X_train_7, label=y_train_7)
xg_train_8 = xgb.DMatrix(X_train_8, label=y_train_8)
xg_train_9 = xgb.DMatrix(X_train_9, label=y_train_9)
xg_train_10 = xgb.DMatrix(X_train_10, label=y_train_10)

xg_test = xgb.DMatrix(X_test, label=y_test)

-initialisation des paramètres et lancement des 10 modèles qui apprennent du précédent via le paramètre xgb_model

In [ ]:

num_round = 100
verbose_eval = 5
watch_list = [(xg_test, 'xg_test')]

params = {'objective': 'reg:squarederror','verbose': False, 'scale_pos_weight':2,'learning_rate':0.1}

print('model 1 \t'); 
model_1 = xgb.train(params, xg_train_1, num_round,watch_list, verbose_eval=verbose_eval)

from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(y_test, model_1.predict(xg_test)))

model_2 = xgb.train(params, xg_train_2, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_1)
del model_1
gc.collect()


model_3 = xgb.train(params, xg_train_3, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_2)
del model_2
gc.collect()


model_4 = xgb.train(params, xg_train_4, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_3)
del model_3
gc.collect()


model_5 = xgb.train(params, xg_train_5, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_4)
del model_4
gc.collect()


model_6 = xgb.train(params, xg_train_6, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_5)
del model_5
gc.collect()


model_7 = xgb.train(params, xg_train_7, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_6)
del model_6
gc.collect()


model_8 = xgb.train(params, xg_train_8, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_7)
del model_7
gc.collect()


model_9 = xgb.train(params, xg_train_9, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_8)
del model_8
gc.collect()

model_10 = xgb.train(params, xg_train_10, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_9)
del model_9
gc.collect()
print('model 10\t', mse(model_10.predict(xg_test), y_test))# "after"

model_10.save_model('gdrive/My Drive/model_full.model')

-on réapplique aux données test

In [ ]:
data = pd.read_csv('gdrive/My Drive/test_V2.csv')

data = data.loc[(data['killPlace'] < 101)]
data = data.drop(["DBNOs","maxPlace","winPoints"], axis = 1)

data.loc[(data['matchType']== 'squad-fpp'),'matchType']='squad'
data.loc[(data['matchType']== 'normal-squad-fpp'),'matchType']='squad'
data.loc[(data['matchType']== 'normal-squad'),'matchType']='squad'
data.loc[(data['matchType']== 'flarefpp'),'matchType']='squad'
data.loc[(data['matchType']== 'flaretpp'),'matchType']='squad'

data.loc[(data['matchType']== 'duo-fpp'),'matchType']='duo'
data.loc[(data['matchType']== 'normal-duo-fpp'),'matchType']='duo'
data.loc[(data['matchType']=='normal-duo'),'matchType']='duo'
data.loc[(data['matchType']=='crashfpp'),'matchType']='duo'
data.loc[(data['matchType']=='crashtpp'),'matchType']='duo'

data.loc[(data['matchType']== 'solo-fpp'),'matchType']='solo'
data.loc[(data['matchType']== 'normal-solo-fpp'),'matchType']='solo'
data.loc[(data['matchType']== 'normal-solo'),'matchType']='solo'

from sklearn.cluster import KMeans
datak = data.drop(["Id","groupId","matchId","matchType"], axis = 1)
kmeans = KMeans(n_clusters=7, random_state=0,max_iter=10,n_init=1).fit(datak)
data['classe']=kmeans.labels_
data['healParDurationM']=data['heals']/data['matchDuration']
data['killParDurationM']=data['kills']/data['matchDuration']
data['killParRideDist']=data['kills'].divide(data['rideDistance'])
data['RideDistParDurationM']=data['rideDistance']/data['matchDuration']
data['BoostParKill']=data['boosts'].divide(data['kills'])
data['KillParSwimDist']=data['kills'].divide(data['swimDistance'])
data['KillParWalkDist']=data['kills'].divide(data['walkDistance'])



ok=data.drop(["matchType","Id","groupId","classe"], axis = 1)
mean=ok.groupby('matchId').mean()
maxi=ok.groupby('matchId').max()


mean.columns = [x + "meanMatch" for x in mean.columns]
maxi.columns = [x + "maxMatch" for x in maxi.columns]




meanG=ok.groupby('groupId').mean()
maxiG=ok.groupby('groupId').max()


meanG.columns = [x + "meanGroupe" for x in meanG.columns]
maxiG.columns = [x + "maxGroupe" for x in maxiG.columns]


data=data.merge(meanG,on='groupId').merge(maxiG,on='groupId').merge(mean,on='matchId').merge(maxi,on='matchId')


data = pd.get_dummies(data, columns=['classe'])
data=data.replace([inf], np.nan)
data=data.fillna(0)
data['DiffKillMaxKill']=data['kills']-data['killsmaxMatch']
data['DiffDmgMaxDmg']=data['damageDealt']-data['damageDealtmaxMatch']


full = data.drop(["Id","groupId","matchId"], axis = 1)
full = pd.get_dummies(full, columns=['matchType'])
data=data.filter(items=['Id'])
full = full.loc[:]
full = xgb.DMatrix(full)
data['WinPlacePerc']=model_10.predict(full)
data=data.filter(items=['Id', 'WinPlacePerc'])

data.loc[(data['WinPlacePerc']<0),'WinPlacePerc']=0
data.loc[(data['WinPlacePerc']>1),'WinPlacePerc']=1
data.to_csv('gdrive/My Drive/SubmitFull.csv',index=False)

## Modèle par type de match

### squad

In [ ]:
squad = squad.drop(["Id","groupId","matchId","matchType"], axis = 1)
squad.reset_index(inplace = True)
feat=squad.iloc[:,np.r_[1:22, 23:150]]
target=squad.iloc[:,[22]]

import pandas as pd
import xgboost as xgb
from sklearn.model_selection import ShuffleSplit
from sklearn.datasets import load_boston
from sklearn.metrics import mean_absolute_error as mse

rs = ShuffleSplit(test_size=0.001, n_splits=1, random_state=0)
for train_idx,test_idx in rs.split(feat):  # this looks silly
    pass


train_split = round(len(train_idx) / 10)
train1_idx = train_idx[0:train_split]
train2_idx = train_idx[train_split:(2*train_split)]
train3_idx = train_idx[(2*train_split):(3*train_split)]
train4_idx = train_idx[(3*train_split):(4*train_split)]
train5_idx = train_idx[(4*train_split):(5*train_split)]
train6_idx = train_idx[(5*train_split):(6*train_split)]
train7_idx = train_idx[(6*train_split):(7*train_split)]
train8_idx = train_idx[(7*train_split):(8*train_split)]
train9_idx = train_idx[(8*train_split):(9*train_split)]
train10_idx = train_idx[(9*train_split):(10*train_split)]

X_train_1 = feat.loc[train1_idx]
X_train_2 = feat.loc[train2_idx]
X_train_3 = feat.loc[train3_idx]
X_train_4 = feat.loc[train4_idx]
X_train_5 = feat.loc[train5_idx]
X_train_6 = feat.loc[train6_idx]
X_train_7 = feat.loc[train7_idx]
X_train_8 = feat.loc[train8_idx]
X_train_9 = feat.loc[train9_idx]
X_train_10 = feat.loc[train10_idx]
X_test=feat.loc[test_idx]

y_train_1 = target.loc[train1_idx]
y_train_2 = target.loc[train2_idx]
y_train_3 = target.loc[train3_idx]
y_train_4 = target.loc[train4_idx]
y_train_5 = target.loc[train5_idx]
y_train_6 = target.loc[train6_idx]
y_train_7 = target.loc[train7_idx]
y_train_8 = target.loc[train8_idx]
y_train_9 = target.loc[train9_idx]
y_train_10 = target.loc[train10_idx]
y_test=target.loc[test_idx]

del [[feat,target]]
gc.collect()

xg_train_1 = xgb.DMatrix(X_train_1, label=y_train_1)
xg_train_2 = xgb.DMatrix(X_train_2, label=y_train_2)
xg_train_3 = xgb.DMatrix(X_train_3, label=y_train_3)
xg_train_4 = xgb.DMatrix(X_train_4, label=y_train_4)
xg_train_5 = xgb.DMatrix(X_train_5, label=y_train_5)
xg_train_6 = xgb.DMatrix(X_train_6, label=y_train_6)
xg_train_7 = xgb.DMatrix(X_train_7, label=y_train_7)
xg_train_8 = xgb.DMatrix(X_train_8, label=y_train_8)
xg_train_9 = xgb.DMatrix(X_train_9, label=y_train_9)
xg_train_10 = xgb.DMatrix(X_train_10, label=y_train_10)

xg_test = xgb.DMatrix(X_test, label=y_test)


num_round = 100
verbose_eval = 5
watch_list = [(xg_test, 'xg_test')]

params = {'objective': 'reg:squarederror','verbose': False, 'scale_pos_weight':2,'learning_rate':0.1}

print('model 1 \t'); 
model_1 = xgb.train(params, xg_train_1, num_round,watch_list, verbose_eval=verbose_eval)

from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(y_test, model_1.predict(xg_test)))


model_2 = xgb.train(params, xg_train_2, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_1)
del model_1
gc.collect()


model_3 = xgb.train(params, xg_train_3, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_2)
del model_2
gc.collect()


model_4 = xgb.train(params, xg_train_4, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_3)
del model_3
gc.collect()


model_5 = xgb.train(params, xg_train_5, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_4)
del model_4
gc.collect()


model_6 = xgb.train(params, xg_train_6, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_5)
del model_5
gc.collect()


model_7 = xgb.train(params, xg_train_7, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_6)
del model_6
gc.collect()


model_8 = xgb.train(params, xg_train_8, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_7)
del model_7
gc.collect()


model_9 = xgb.train(params, xg_train_9, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_8)
del model_8
gc.collect()


model_squad = xgb.train(params, xg_train_10, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_9)
del model_9
gc.collect()
print('model 10\t', mse(model_squad.predict(xg_test), y_test))# "after"


### duo

In [ ]:
duo = duo.drop(["Id","groupId","matchId","matchType"], axis = 1)
duo.reset_index(inplace = True)
feat=duo.iloc[:,np.r_[1:22, 23:150]]
target=duo.iloc[:,[22]]

import pandas as pd
import xgboost as xgb
from sklearn.model_selection import ShuffleSplit
from sklearn.datasets import load_boston
from sklearn.metrics import mean_absolute_error as mse

rs = ShuffleSplit(test_size=0.001, n_splits=1, random_state=0)
for train_idx,test_idx in rs.split(feat):  # this looks silly
    pass


train_split = round(len(train_idx) / 10)
train1_idx = train_idx[0:train_split]
train2_idx = train_idx[train_split:(2*train_split)]
train3_idx = train_idx[(2*train_split):(3*train_split)]
train4_idx = train_idx[(3*train_split):(4*train_split)]
train5_idx = train_idx[(4*train_split):(5*train_split)]
train6_idx = train_idx[(5*train_split):(6*train_split)]
train7_idx = train_idx[(6*train_split):(7*train_split)]
train8_idx = train_idx[(7*train_split):(8*train_split)]
train9_idx = train_idx[(8*train_split):(9*train_split)]
train10_idx = train_idx[(9*train_split):(10*train_split)]

X_train_1 = feat.loc[train1_idx]
X_train_2 = feat.loc[train2_idx]
X_train_3 = feat.loc[train3_idx]
X_train_4 = feat.loc[train4_idx]
X_train_5 = feat.loc[train5_idx]
X_train_6 = feat.loc[train6_idx]
X_train_7 = feat.loc[train7_idx]
X_train_8 = feat.loc[train8_idx]
X_train_9 = feat.loc[train9_idx]
X_train_10 = feat.loc[train10_idx]
X_test=feat.loc[test_idx]

y_train_1 = target.loc[train1_idx]
y_train_2 = target.loc[train2_idx]
y_train_3 = target.loc[train3_idx]
y_train_4 = target.loc[train4_idx]
y_train_5 = target.loc[train5_idx]
y_train_6 = target.loc[train6_idx]
y_train_7 = target.loc[train7_idx]
y_train_8 = target.loc[train8_idx]
y_train_9 = target.loc[train9_idx]
y_train_10 = target.loc[train10_idx]
y_test=target.loc[test_idx]

del [[feat,target]]
gc.collect()

xg_train_1 = xgb.DMatrix(X_train_1, label=y_train_1)
xg_train_2 = xgb.DMatrix(X_train_2, label=y_train_2)
xg_train_3 = xgb.DMatrix(X_train_3, label=y_train_3)
xg_train_4 = xgb.DMatrix(X_train_4, label=y_train_4)
xg_train_5 = xgb.DMatrix(X_train_5, label=y_train_5)
xg_train_6 = xgb.DMatrix(X_train_6, label=y_train_6)
xg_train_7 = xgb.DMatrix(X_train_7, label=y_train_7)
xg_train_8 = xgb.DMatrix(X_train_8, label=y_train_8)
xg_train_9 = xgb.DMatrix(X_train_9, label=y_train_9)
xg_train_10 = xgb.DMatrix(X_train_10, label=y_train_10)

xg_test = xgb.DMatrix(X_test, label=y_test)


num_round = 100
verbose_eval = 5
watch_list = [(xg_test, 'xg_test')]

params = {'objective': 'reg:squarederror','verbose': False, 'scale_pos_weight':2,'learning_rate':0.1}

print('model 1 \t'); 
model_1 = xgb.train(params, xg_train_1, num_round,watch_list, verbose_eval=verbose_eval)

from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(y_test, model_1.predict(xg_test)))


model_2 = xgb.train(params, xg_train_2, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_1)
del model_1
gc.collect()


model_3 = xgb.train(params, xg_train_3, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_2)
del model_2
gc.collect()


model_4 = xgb.train(params, xg_train_4, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_3)
del model_3
gc.collect()


model_5 = xgb.train(params, xg_train_5, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_4)
del model_4
gc.collect()


model_6 = xgb.train(params, xg_train_6, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_5)
del model_5
gc.collect()


model_7 = xgb.train(params, xg_train_7, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_6)
del model_6
gc.collect()


model_8 = xgb.train(params, xg_train_8, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_7)
del model_7
gc.collect()


model_9 = xgb.train(params, xg_train_9, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_8)
del model_8
gc.collect()


model_duo = xgb.train(params, xg_train_10, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_9)
del model_9
gc.collect()
print('model 10\t', mse(model_duo.predict(xg_test), y_test))# "after"

### solo

In [ ]:
solo = solo.drop(["Id","groupId","matchId","matchType"], axis = 1)
solo.reset_index(inplace = True)
feat=solo.iloc[:,np.r_[1:22, 23:150]]
target=solo.iloc[:,[22]]

import pandas as pd
import xgboost as xgb
from sklearn.model_selection import ShuffleSplit
from sklearn.datasets import load_boston
from sklearn.metrics import mean_absolute_error as mse

rs = ShuffleSplit(test_size=0.001, n_splits=1, random_state=0)
for train_idx,test_idx in rs.split(feat):  # this looks silly
    pass


train_split = round(len(train_idx) / 10)
train1_idx = train_idx[0:train_split]
train2_idx = train_idx[train_split:(2*train_split)]
train3_idx = train_idx[(2*train_split):(3*train_split)]
train4_idx = train_idx[(3*train_split):(4*train_split)]
train5_idx = train_idx[(4*train_split):(5*train_split)]
train6_idx = train_idx[(5*train_split):(6*train_split)]
train7_idx = train_idx[(6*train_split):(7*train_split)]
train8_idx = train_idx[(7*train_split):(8*train_split)]
train9_idx = train_idx[(8*train_split):(9*train_split)]
train10_idx = train_idx[(9*train_split):(10*train_split)]

X_train_1 = feat.loc[train1_idx]
X_train_2 = feat.loc[train2_idx]
X_train_3 = feat.loc[train3_idx]
X_train_4 = feat.loc[train4_idx]
X_train_5 = feat.loc[train5_idx]
X_train_6 = feat.loc[train6_idx]
X_train_7 = feat.loc[train7_idx]
X_train_8 = feat.loc[train8_idx]
X_train_9 = feat.loc[train9_idx]
X_train_10 = feat.loc[train10_idx]
X_test=feat.loc[test_idx]

y_train_1 = target.loc[train1_idx]
y_train_2 = target.loc[train2_idx]
y_train_3 = target.loc[train3_idx]
y_train_4 = target.loc[train4_idx]
y_train_5 = target.loc[train5_idx]
y_train_6 = target.loc[train6_idx]
y_train_7 = target.loc[train7_idx]
y_train_8 = target.loc[train8_idx]
y_train_9 = target.loc[train9_idx]
y_train_10 = target.loc[train10_idx]
y_test=target.loc[test_idx]

del [[feat,target]]
gc.collect()

xg_train_1 = xgb.DMatrix(X_train_1, label=y_train_1)
xg_train_2 = xgb.DMatrix(X_train_2, label=y_train_2)
xg_train_3 = xgb.DMatrix(X_train_3, label=y_train_3)
xg_train_4 = xgb.DMatrix(X_train_4, label=y_train_4)
xg_train_5 = xgb.DMatrix(X_train_5, label=y_train_5)
xg_train_6 = xgb.DMatrix(X_train_6, label=y_train_6)
xg_train_7 = xgb.DMatrix(X_train_7, label=y_train_7)
xg_train_8 = xgb.DMatrix(X_train_8, label=y_train_8)
xg_train_9 = xgb.DMatrix(X_train_9, label=y_train_9)
xg_train_10 = xgb.DMatrix(X_train_10, label=y_train_10)

xg_test = xgb.DMatrix(X_test, label=y_test)


num_round = 100
verbose_eval = 5
watch_list = [(xg_test, 'xg_test')]

params = {'objective': 'reg:squarederror','verbose': False, 'scale_pos_weight':2,'learning_rate':0.1}

print('model 1 \t'); 
model_1 = xgb.train(params, xg_train_1, num_round,watch_list, verbose_eval=verbose_eval)

from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(y_test, model_1.predict(xg_test)))


model_2 = xgb.train(params, xg_train_2, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_1)
del model_1
gc.collect()


model_3 = xgb.train(params, xg_train_3, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_2)
del model_2
gc.collect()


model_4 = xgb.train(params, xg_train_4, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_3)
del model_3
gc.collect()


model_5 = xgb.train(params, xg_train_5, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_4)
del model_4
gc.collect()


model_6 = xgb.train(params, xg_train_6, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_5)
del model_5
gc.collect()


model_7 = xgb.train(params, xg_train_7, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_6)
del model_6
gc.collect()


model_8 = xgb.train(params, xg_train_8, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_7)
del model_7
gc.collect()


model_9 = xgb.train(params, xg_train_9, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_8)
del model_8
gc.collect()


model_solo = xgb.train(params, xg_train_10, num_round,watch_list, verbose_eval=verbose_eval, xgb_model=model_9)
del model_9
gc.collect()
print('model 10\t', mse(model_solo.predict(xg_test), y_test))# "after"

on réapplique aux données test

In [ ]:
data = pd.read_csv('gdrive/My Drive/test_V2.csv')

data = data.loc[(data['killPlace'] < 101)]
data = data.drop(["DBNOs","maxPlace","winPoints"], axis = 1)

data.loc[(data['matchType']== 'squad-fpp'),'matchType']='squad'
data.loc[(data['matchType']== 'normal-squad-fpp'),'matchType']='squad'
data.loc[(data['matchType']== 'normal-squad'),'matchType']='squad'
data.loc[(data['matchType']== 'flarefpp'),'matchType']='squad'
data.loc[(data['matchType']== 'flaretpp'),'matchType']='squad'

data.loc[(data['matchType']== 'duo-fpp'),'matchType']='duo'
data.loc[(data['matchType']== 'normal-duo-fpp'),'matchType']='duo'
data.loc[(data['matchType']=='normal-duo'),'matchType']='duo'
data.loc[(data['matchType']=='crashfpp'),'matchType']='duo'
data.loc[(data['matchType']=='crashtpp'),'matchType']='duo'

data.loc[(data['matchType']== 'solo-fpp'),'matchType']='solo'
data.loc[(data['matchType']== 'normal-solo-fpp'),'matchType']='solo'
data.loc[(data['matchType']== 'normal-solo'),'matchType']='solo'

from sklearn.cluster import KMeans
datak = data.drop(["Id","groupId","matchId","matchType"], axis = 1)
kmeans = KMeans(n_clusters=7, random_state=0,max_iter=10,n_init=1).fit(datak)
data['classe']=kmeans.labels_
data['healParDurationM']=data['heals']/data['matchDuration']
data['killParDurationM']=data['kills']/data['matchDuration']
data['killParRideDist']=data['kills'].divide(data['rideDistance'])
data['RideDistParDurationM']=data['rideDistance']/data['matchDuration']
data['BoostParKill']=data['boosts'].divide(data['kills'])
data['KillParSwimDist']=data['kills'].divide(data['swimDistance'])
data['KillParWalkDist']=data['kills'].divide(data['walkDistance'])


ok=data.drop(["matchType","Id","groupId","classe"], axis = 1)
mean=ok.groupby('matchId').mean()
maxi=ok.groupby('matchId').max()


mean.columns = [x + "meanMatch" for x in mean.columns]
maxi.columns = [x + "maxMatch" for x in maxi.columns]




meanG=ok.groupby('groupId').mean()
maxiG=ok.groupby('groupId').max()


meanG.columns = [x + "meanGroupe" for x in meanG.columns]
maxiG.columns = [x + "maxGroupe" for x in maxiG.columns]


data=data.merge(meanG,on='groupId').merge(maxiG,on='groupId').merge(mean,on='matchId').merge(maxi,on='matchId')


data = pd.get_dummies(data, columns=['classe'])
data=data.replace([inf], np.nan)
data=data.fillna(0)
data['DiffKillMaxKill']=data['kills']-data['killsmaxMatch']
data['DiffDmgMaxDmg']=data['damageDealt']-data['damageDealtmaxMatch']

solo=data[(data.matchType=='solo')]
duo=data[(data.matchType=='duo')]
squad=data[(data.matchType=='squad')]

squadtest = squad.drop(["Id","groupId","matchId","matchType"], axis = 1)
squadtest.reset_index(inplace = True)
del squadtest['index']
squadtest = squadtest.loc[:]
squadtest = xgb.DMatrix(squadtest)
squad['WinPlacePerc']=model_squad.predict(squadtest)
squad=squad.filter(items=['Id', 'WinPlacePerc'])

duotest = duo.drop(["Id","groupId","matchId","matchType"], axis = 1)
duotest.reset_index(inplace = True)
del duotest['index']
duotest = duotest.loc[:]
duotest = xgb.DMatrix(duotest)
duo['WinPlacePerc']=model_duo.predict(duotest)
duo=duo.filter(items=['Id', 'WinPlacePerc'])

solotest = solo.drop(["Id","groupId","matchId","matchType"], axis = 1)
solotest.reset_index(inplace = True)
del solotest['index']
solotest = solotest.loc[:]
solotest = xgb.DMatrix(solotest)
solo['WinPlacePerc']=model_solo.predict(solotest)
solo=solo.filter(items=['Id', 'WinPlacePerc'])

data=pd.concat([squad, duo,solo], ignore_index=True)
data.loc[(data['WinPlacePerc']<0),'WinPlacePerc']=0
data.loc[(data['WinPlacePerc']>1),'WinPlacePerc']=1
data.to_csv('gdrive/My Drive/Submit_teamMatch.csv',index=False)